In [9]:
import numpy as np
# from importnb import Notebook
# with Notebook(): 
#     import reproduccion_ulutas as rp

import reproduccion_ulutas as rp

## Parameters

In [10]:
POPULATION_SIZE = 100   # indicate the initial size of antibodies population
# N is a random % of antibodies with highest affinities selected to be cloned
R = 1 # % of best cloned antibodies to the pool of antibodies
B = 0.1 # worst % of the whole population (RECEPTOR EDITING)

# dataset = 'data/instances/testset_a/5x7_King_Nakornchai(1982)[Figure-1a].txt'
# dataset = 'data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt'
dataset = 'data/instances/testset_a/5x18_Seifoddini(1989b).txt'
# dataset = 'data/instances/testset_a/6x8_Kusiak_Cho(1992).txt'
# dataset = 'data/instances/testset_a/7x11_Boctor(1991)[Example-1].txt' #A6
# dataset = 'data/instances/testset_a/8x20_Chandrasekharan_Rajagopalan(1986a).txt' #A8




# dataset = 'data/instances/testset_a/30x50_Stanfel(1985)[Figure-5].txt'

MAX_ITERATIONS = 1000
MAX_ITERATIONS_WITHOUT_IMPROVEMENT = 150
log = False # Boolean: print logs at the execution of the algorithm.

## Algorithm

In [11]:
#reads data and translates it into part-machine matrix
matrix, m, p,number_of_operations, columns, rows = rp.part_machine_incidence_matrix(dataset)
if log == True: print("\nPart-machine matrix\n",matrix)
#GENERATION OF INITIAL POPULATION
antibodies = rp.generation_initial_population(p = p, m= m, population_size = POPULATION_SIZE)
if log == True: print("\nInitial antibodies\n",antibodies)

iterations, best_solution, improvement_counter = 0, 0, 0

while iterations <= MAX_ITERATIONS:
    if log == True: print("\n Iteration number",iterations+1)
    #identifies cells in the antibodies
    total_cells = rp.cell_identification(antibodies = antibodies)
    if log == True: print("\n Cells\n", total_cells)

    #translates numbers into parts and machines
    total_machines, total_parts = rp.decode_cells(total_cells = total_cells, rows= rows, columns = columns, p = p, log = log) 

    #calculates part-machine matrix for each antibody
    antibody_matrices = rp.create_machine_part_matrix(matrix = matrix, 
                                                    antibodies = antibodies, 
                                                    total_machines = total_machines, 
                                                    total_parts = total_parts)
    #evaluates efficacie of each antibody
    efficacies, affinities, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, 
                                                                        total_cells = total_cells,
                                                                        number_of_operations = number_of_operations)
    if log == True: print("\n Efficacies",efficacies)
    if log == True: print("\n Affinities",affinities)

    # SELECTS N% OF ANTIBODIES WITH HIGHES ANTIBODIES (SELECTION ROULETTE WHEEL)
    N = np.random.rand(1) # N is a random number each iteration
    if log: print("N% =",N[0])
    cloned_antibodies, positions_antibodies_selected = rp.antibodies_selection_v3(antibodies=antibodies, N=N[0], affinities = affinities)
    if log == True: print("\n Cloned antibodies\n",cloned_antibodies)
    if log == True: print("\n Cloned antibodies positions (pool)\n",positions_antibodies_selected)
    
    # MUTATION
    rp.mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies, log= log)
    if log == True: print("\n Mutation of cloned antibodies\n",cloned_antibodies)

    #CLONES: identifies cells in the antibody
    cloned_total_cells = rp.cell_identification(cloned_antibodies)
    #CLONES: translates numbers into parts and machines
    cloned_total_machines, cloned_total_parts = rp.decode_cells(total_cells = cloned_total_cells,rows= rows, columns = columns, p = p, log = log)
    #CLONES: calculates part-machine matrix for each antibody
    cloned_antibody_matrices = rp.create_machine_part_matrix(matrix = matrix,
                                                                antibodies = cloned_antibodies, 
                                                                total_machines = cloned_total_machines, 
                                                                total_parts = cloned_total_parts)                                                          
    #CLONES evaluates efficacie of each antibody
    cloned_efficacies, cloned_affinities, cloned_voids, cloned_exceptions = rp.evaluate_antibodies(cloned_antibody_matrices,
                                                                                                    cloned_total_cells,
                                                                                                    number_of_operations)
    if log == True: print("\n Cloned Efficacies",cloned_efficacies)
    if log == True: print("\n Cloned Affinities",cloned_affinities)
    # Add R% of best cloned antibodies to the pool of antibodies
    antibodies, efficacies, affinities = rp.select_best_cloned_antibodies_v2(antibodies = antibodies,
                                        cloned_antibodies = cloned_antibodies,
                                        efficacies = efficacies, 
                                        cloned_efficacies = cloned_efficacies,
                                        affinities = affinities, 
                                        cloned_affinities = cloned_affinities,
                                        R = R,
                                        positions_antibodies_selected = positions_antibodies_selected, 
                                        log = log)
    if log == True: print("\n New antibodies pool \n",antibodies)                                                                        
    # RECEPTOR EDITING: Remove worst members of the antibodies pool
    antibodies, efficacies, affinities, amount_antibodies_erased = rp.receptor_editing(antibodies_pool = antibodies, 
                                                                                efficacies = efficacies, 
                                                                                affinities=affinities, 
                                                                                B = B, 
                                                                                log = log)
    if log == True: print("\n Antibodies pool after removal \n", antibodies)
    #GENERATION OF NEW RANDOM ANTIBODIES
    number_new_random_antibodies = amount_antibodies_erased
    new_random_antibodies = rp.generation_initial_population(p = p, 
                                                        m = m, 
                                                        population_size = number_new_random_antibodies)
    if log == True: print("\n New random antibodies \n",new_random_antibodies)                                                        
    antibodies = np.concatenate((antibodies, new_random_antibodies), axis = 0)
    if log == True: print("\n Antibodies pool after adding new random antibodies \n",antibodies)
    
    #TERMINATION CRITERIA AND SELECTION OF BEST SOLUTION
    iteration_best_solution = np.amax(efficacies) #max value efficacy for this iteration
    index_best_solution = np.argmax(efficacies)
    best_antibody = antibodies[index_best_solution]
    if log: print("\n Best solution at the moment is {} (antibody {}, index {})".format(iteration_best_solution,best_antibody,index_best_solution))
    if iteration_best_solution <= best_solution:
        improvement_counter = improvement_counter + 1
        if improvement_counter >= MAX_ITERATIONS_WITHOUT_IMPROVEMENT:
            print("Maximun number of iterations without improvement reached: {}".format(MAX_ITERATIONS_WITHOUT_IMPROVEMENT))
            print("Best solution obtained is {}".format(best_solution))
            break
    else:
        best_solution = iteration_best_solution
        improvement_counter = 0
    #update iteration counter
    iterations = iterations + 1

Maximun number of iterations without improvement reached: 150
Best solution obtained is 0.7959183673469388


In [12]:
iterations

550

In [13]:
best_solution

0.7959183673469388

In [14]:
best_antibody

array([ 0,  0,  7, 10,  4, 21, 23, 15, 18, 20,  9,  0,  0, 12, 14, 19, 16,
       13,  1,  8,  5, 22, 17,  6,  2,  3, 11])

In [15]:
cell_identified_best_antibody = rp.cell_identification([best_antibody])
total_machines, total_parts = rp.decode_cells(total_cells = cell_identified_best_antibody, rows= rows, columns = columns, p = p, log = log)

antibody_matrices = rp.create_machine_part_matrix(matrix=matrix,
                                                antibodies=[best_antibody], 
                                                total_machines=total_machines, 
                                                total_parts=total_parts)
antibody_matrices                                                

[     M3  M5  M2  M1  M4
 P7    1   0   1   0   0
 P10   1   1   1   0   0
 P4    1   1   1   0   0
 P15   1   1   1   0   0
 P18   1   1   1   0   0
 P9    0   1   0   0   0
 P12   0   0   1   1   1
 P14   0   0   0   1   1
 P16   0   0   0   1   1
 P13   0   0   1   1   1
 P1    0   0   1   1   1
 P8    0   0   1   1   1
 P5    0   0   0   1   1
 P17   0   0   0   1   1
 P6    0   0   1   1   1
 P2    0   0   0   1   1
 P3    0   0   1   1   1
 P11   0   0   1   1   1]

In [16]:
efficacies, affinities, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, 
                                                                    total_cells=cell_identified_best_antibody,
                                                                    number_of_operations = number_of_operations)
efficacies

[0.7959183673469388]